In [68]:
#Importacion ded librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

In [69]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
# Establecer ruta de acceso en dr
import os
print(os.getcwd())
os.chdir("/content/gdrive/MyDrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive


In [70]:
#Importacion de los datos
dataset = pd.read_csv("insurance.csv", sep = ",")

In [71]:
#Vemos el dataset
dataset

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [72]:
dataset.shape

(1338, 7)

### One Hot Encoding

Primer paso, pasar columna 'Sex' a numero

In [73]:
dataset.sex.value_counts()

male      676
female    662
Name: sex, dtype: int64

In [74]:
dataset.sex = dataset.sex.map({'male': 0,'female': 1})

In [75]:
print(dataset)

      age  sex     bmi  children smoker     region      charges
0      19    1  27.900         0    yes  southwest  16884.92400
1      18    0  33.770         1     no  southeast   1725.55230
2      28    0  33.000         3     no  southeast   4449.46200
3      33    0  22.705         0     no  northwest  21984.47061
4      32    0  28.880         0     no  northwest   3866.85520
...   ...  ...     ...       ...    ...        ...          ...
1333   50    0  30.970         3     no  northwest  10600.54830
1334   18    1  31.920         0     no  northeast   2205.98080
1335   18    1  36.850         0     no  southeast   1629.83350
1336   21    1  25.800         0     no  southwest   2007.94500
1337   61    1  29.070         0    yes  northwest  29141.36030

[1338 rows x 7 columns]


Segundo paso, pasar columna 'smoker' a numero

In [76]:
dataset.smoker = dataset.smoker.map({'yes': 1,'no': 0})

Tercer paso, pasar columna 'region' a numero

In [77]:
dataset.region.value_counts()

southeast    364
southwest    325
northwest    325
northeast    324
Name: region, dtype: int64

In [78]:
pd.get_dummies(dataset.region)

,northeast,northwest,southeast,southwest
0,0,0,0,1
1,0,0,1,0
2,0,0,1,0
3,0,1,0,0
4,0,1,0,0
...,...,...,...,...
1333,0,1,0,0
1334,1,0,0,0
1335,0,0,1,0
1336,0,0,0,1


In [79]:
dataset = dataset.join(pd.get_dummies(dataset.region))
dataset = dataset.drop('region',axis = 1)

In [80]:
dataset.head()

,age,sex,bmi,children,smoker,charges,northeast,northwest,southeast,southwest
0,19,1,27.900,0,1,16884.92400,0,0,0,1
1,18,0,33.770,1,0,1725.55230,0,0,1,0
2,28,0,33.000,3,0,4449.46200,0,0,1,0
3,33,0,22.705,0,0,21984.47061,0,1,0,0
4,32,0,28.880,0,0,3866.85520,0,1,0,0


In [81]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        1338 non-null   int64  
 1   sex        1338 non-null   int64  
 2   bmi        1338 non-null   float64
 3   children   1338 non-null   int64  
 4   smoker     1338 non-null   int64  
 5   charges    1338 non-null   float64
 6   northeast  1338 non-null   uint8  
 7   northwest  1338 non-null   uint8  
 8   southeast  1338 non-null   uint8  
 9   southwest  1338 non-null   uint8  
dtypes: float64(2), int64(4), uint8(4)
memory usage: 68.1 KB


Ahora que todos los campos son numericos, podemos proceder al modelado

### Modelado

In [42]:
dataset.describe()

,age,sex,bmi,children,smoker,charges,northeast,northwest,southeast,southwest
count,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,0.494768,30.663397,1.094918,0.204783,13270.422265,0.242152,0.242900,0.272048,0.242900
std,14.049960,0.500160,6.098187,1.205493,0.403694,12110.011237,0.428546,0.428995,0.445181,0.428995
min,18.000000,0.000000,15.960000,0.000000,0.000000,1121.873900,0.000000,0.000000,0.000000,0.000000
25%,27.000000,0.000000,26.296250,0.000000,0.000000,4740.287150,0.000000,0.000000,0.000000,0.000000
50%,39.000000,0.000000,30.400000,1.000000,0.000000,9382.033000,0.000000,0.000000,0.000000,0.000000
75%,51.000000,1.000000,34.693750,2.000000,0.000000,16639.912515,0.000000,0.000000,1.000000,0.000000
max,64.000000,1.000000,53.130000,5.000000,1.000000,63770.428010,1.000000,1.000000,1.000000,1.000000


# Primer modelo
Se realizara una prueba con 3 variables iniciales, la edad, el bmi y smoker

In [82]:
# 1 )Preparación de datos
X = dataset[['age', 'bmi','smoker']]
y = dataset['charges']

In [83]:
# 2) Separacion en train y test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [84]:
#Entrenamiento del modelo
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [85]:
# 'age', 'sex', 'bmi','children','smoker','region'
regressor.coef_

array([  259.41020462,   326.45023215, 23675.37184666])

In [86]:
regressor.intercept_

-11707.802334583159

In [87]:
X.columns

Index(['age', 'bmi', 'smoker'], dtype='object')

In [88]:
coeff_df = pd.DataFrame(regressor.coef_, X.columns, columns=['Coefficient'])
coeff_df

,Coefficient
age,259.410205
bmi,326.450232
smoker,23675.371847


In [91]:
#Realizando las predicciones
y_pred = regressor.predict(X_test)

In [92]:
y_test

764      9095.06825
887      5272.17580
890     29330.98315
1293     9301.89355
259     33750.29180
           ...     
109     47055.53210
575     12222.89830
535      6067.12675
543     63770.42801
846      9872.70100
Name: charges, Length: 268, dtype: float64

In [93]:
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df['Sesgo']=df.Actual -df.Predicted
df['Error_porc']=((df.Actual -df.Predicted)/df.Actual) *100
df

,Actual,Predicted,Sesgo,Error_porc
764,9095.06825,8184.041468,911.026782,10.016712
887,5272.17580,7431.001001,-2158.825201,-40.947519
890,29330.98315,37346.437099,-8015.453949,-27.327601
1293,9301.89355,8629.528305,672.365245,7.228262
259,33750.29180,27316.654810,6433.636990,19.062463
...,...,...,...,...
109,47055.53210,39765.551050,7289.981050,15.492293
575,12222.89830,12207.642341,15.255959,0.124815
535,6067.12675,7298.553197,-1231.426447,-20.296699
543,63770.42801,41452.726068,22317.701942,34.996946


# Evaluacion del modelo 1

In [95]:
#Evaluación de Modelos
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 4260.560091099392
Mean Squared Error: 34512843.8802279
Root Mean Squared Error: 5874.763304187489


In [96]:
from sklearn.metrics import r2_score
print('El r^2 es:',r2_score(y_test,y_pred))

El r^2 es: 0.7776932310583374


# Modelo # 2


In [97]:
# 1 )Preparación de datos
X = dataset[['age', 'bmi','smoker','children','sex','northeast','northwest','southeast','southwest']]
y = dataset['charges']

In [98]:
# 2) Separacion en train y test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [99]:
#Entrenamiento del modelo
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [100]:
# 'age', 'bmi','smoker','children','sex','northeast','northwest','southeast','southwest'
regressor.coef_

array([ 2.56975706e+02,  3.37092552e+02,  2.36511289e+04,  4.25278784e+02,
        1.85916916e+01,  4.59585244e+02,  8.89079180e+01, -1.98279052e+02,
       -3.50214110e+02])

In [101]:
regressor.intercept_

-12409.395986211335

In [102]:
coeff_df = pd.DataFrame(regressor.coef_, X.columns, columns=['Coefficient'])
coeff_df

,Coefficient
age,256.975706
bmi,337.092552
smoker,23651.128856
children,425.278784
sex,18.591692
northeast,459.585244
northwest,88.907918
southeast,-198.279052
southwest,-350.214110


In [ ]:
#Realizando las predicciones
y_pred = regressor.predict(X_test)

# Evaluacion del modelo 2

In [104]:
#Evaluación de Modelos
from sklearn import metrics
import numpy as np
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 4181.194473753641
Mean Squared Error: 33596915.85136146
Root Mean Squared Error: 5796.2846592762735


In [105]:
from sklearn.metrics import r2_score
print('El r^2 es:',r2_score(y_test,y_pred))

El r^2 es: 0.7835929767120723


#Conclusiones:

De acuerdo al r^2, se observa que el modelo 1 tiene un mayor coeficiente que el modelo 2, lo que puede indicar que el 2 es mejor modelo que el 1 para estimar los cargos del sistema de saud. Sin embargo, sabemos por experiencia que el coeficiente de determinacion suele aumentar falsamente cuando aumentamos el # de variables, por lo cul decidimos utilizar otros indicadores para comparar ambos modelos.


Partiendo del supuesto de que nuestro dataset no tiene datos atipicos, encontramos que el MAE, MSE y el RMSE son menores en el modelo 2 vs el 1, lo que confirma que este segundo modelo tiene una mejor predicción que el primero, por lo cual concluimos que es mejor trabajar con todas las variables disponibles en lugar de las tres que escogimos inicialmente.